In [14]:
import requests
import pandas as pd

# Utilisation de pd.set_option pour afficher toutes les colonnes du DataFrame
pd.set_option('display.max_columns', None)

station_names = ["Paris Bercy", "Paris Saint-Lazare", "Paris Gare de Lyon", "Paris Gare du Nord", "Paris Montparnasse", "Paris Est", "Paris Austerlitz"]

records = []

for name in station_names:
    for year in range(2019, 2023):
        url = f"https://ressources.data.sncf.com/api/records/1.0/search/?dataset=objets-trouves-restitution&q=Paris&rows=10000&sort=date&facet=date&facet=gc_obo_gare_origine_r_name&facet=gc_obo_type_c&refine.gc_obo_gare_origine_r_name={name.replace(' ', '+')}&refine.date={year}"
        response = requests.get(url)
        data = response.json()
        records += data['records']

df1 = pd.json_normalize(records)
df1 = df1.loc[:, ['fields.date', 'fields.gc_obo_gare_origine_r_name', 'fields.gc_obo_type_c']]
df1['fields.date'] = pd.to_datetime(df1['fields.date']).dt.strftime('%Y-%m-%d')
# création du CSV 
df1.to_csv("objets-trouves.csv", index=False)


In [15]:
df1

,fields.date,fields.gc_obo_gare_origine_r_name,fields.gc_obo_type_c
0,2019-12-29,Paris Bercy,"Porte-monnaie / portefeuille, argent, titres"
1,2019-12-26,Paris Bercy,"Bagagerie: sacs, valises, cartables"
2,2019-12-26,Paris Bercy,"Bagagerie: sacs, valises, cartables"
3,2019-12-26,Paris Bercy,"Bagagerie: sacs, valises, cartables"
4,2019-12-26,Paris Bercy,"Vêtements, chaussures"
...,...,...,...
81627,2022-01-04,Paris Austerlitz,"Porte-monnaie / portefeuille, argent, titres"
81628,2022-01-04,Paris Austerlitz,"Porte-monnaie / portefeuille, argent, titres"
81629,2022-01-04,Paris Austerlitz,"Bagagerie: sacs, valises, cartables"
81630,2022-01-04,Paris Austerlitz,"Bagagerie: sacs, valises, cartables"


In [16]:
import sqlite3
connexion= sqlite3.connect("bdd.db")
curseur = connexion.cursor()
curseur.execute('''CREATE TABLE IF NOT EXISTS df1 (
              date TEXT NOT NULL,
              gare TEXT NOT NULL,
              feels_like FLOAT,
              type_objet TEXT
              );''')

connexion.commit()

query = "INSERT INTO df1 (date, gare, type_objet) VALUES (?, ?, ?)"
curseur.executemany(query, df1.values.tolist())

# Validation de la transaction
connexion.commit()

# Fermeture de la connexion
connexion.close()

In [17]:
# # URL pour les données de température de Paris
# url = "https://votre_url_de_donnees_de_temperature"

# # Effectuer une requête HTTP pour obtenir les données en format JSON
# response = requests.get(url)
# data = response.json()

# # Charger les données JSON dans un DataFrame
# df = pd.json_normalize(data)

# # Filtrer les données pour la période 2019-2022
# df = df[(df['annee'] >= 2019) & (df['annee'] <= 2022)]

# # Afficher les premières lignes du DataFrame
# print(df.head())

In [18]:
df1['fields.date'] = pd.to_datetime(df1['fields.date'])
df1_2019 = df1[df1['fields.date'].dt.year == 2019]
df1_2019

,fields.date,fields.gc_obo_gare_origine_r_name,fields.gc_obo_type_c
0,2019-12-29,Paris Bercy,"Porte-monnaie / portefeuille, argent, titres"
1,2019-12-26,Paris Bercy,"Bagagerie: sacs, valises, cartables"
2,2019-12-26,Paris Bercy,"Bagagerie: sacs, valises, cartables"
3,2019-12-26,Paris Bercy,"Bagagerie: sacs, valises, cartables"
4,2019-12-26,Paris Bercy,"Vêtements, chaussures"
...,...,...,...
80380,2019-01-03,Paris Austerlitz,"Appareils électroniques, informatiques, appare..."
80381,2019-01-03,Paris Austerlitz,"Clés, porte-clés, badge magnétique"
80382,2019-01-03,Paris Austerlitz,"Bagagerie: sacs, valises, cartables"
80383,2019-01-03,Paris Austerlitz,"Appareils électroniques, informatiques, appare..."


In [19]:
#df_provisoire_2019['fields.date'] = pd.to_datetime(df_provisoire_['fields.date'])
df1_2019['fields.date']=df1_2019['fields.date'].dt.strftime('%U')

/tmp/ipykernel_282436/2308099575.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_2019['fields.date']=df1_2019['fields.date'].dt.strftime('%U')


In [20]:
df1_2019

,fields.date,fields.gc_obo_gare_origine_r_name,fields.gc_obo_type_c
0,52,Paris Bercy,"Porte-monnaie / portefeuille, argent, titres"
1,51,Paris Bercy,"Bagagerie: sacs, valises, cartables"
2,51,Paris Bercy,"Bagagerie: sacs, valises, cartables"
3,51,Paris Bercy,"Bagagerie: sacs, valises, cartables"
4,51,Paris Bercy,"Vêtements, chaussures"
...,...,...,...
80380,00,Paris Austerlitz,"Appareils électroniques, informatiques, appare..."
80381,00,Paris Austerlitz,"Clés, porte-clés, badge magnétique"
80382,00,Paris Austerlitz,"Bagagerie: sacs, valises, cartables"
80383,00,Paris Austerlitz,"Appareils électroniques, informatiques, appare..."


In [21]:
df1_2019.to_csv("objets-trouves_2019.csv", index=False)


In [22]:
import matplotlib 
df1_2019['fields.date'].value_counts()

28    677
37    645
26    627
34    612
27    597
44    594
35    593
22    584
25    579
06    576
40    576
11    567
15    567
24    563
32    562
08    560
23    554
33    552
12    551
42    547
29    541
31    540
09    537
16    535
05    532
19    526
18    526
20    524
13    519
03    511
46    509
47    504
43    504
21    496
07    483
17    482
36    481
38    479
14    472
41    459
01    456
30    450
02    450
04    449
45    448
10    436
39    422
48    371
00    364
50    283
51    259
49    188
52    142
Name: fields.date, dtype: int64